## ChatGPT with LangChain

This notebook provides a quick introduction to ChatGPT and related features supported in LangChain.

Install these libraries before getting started. Ideally, you want to create a dedicated environment for this.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

Load environment variables. You can use anything you like but I used `python-dotenv`. Just create a `.env` file with your `OPENAI_API_KEY` then load it as follows:

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

We are adapting code from [here](https://langchain.readthedocs.io/en/latest/modules/chat/getting_started.html).

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [5]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

/Users/elvissaravia/opt/miniconda3/envs/peguide/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


ChatGPT support different types of messages identifiable by the role. LangChain. Recall how we make a basic call to ChatGPT using `openai`? Here is an example:

```python
MODEL = "gpt-3.5-turbo"

response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an AI research assistant. You use a tone that is technical and scientific."},
        {"role": "user", "content": "Hello, who are you?"},
        {"role": "assistant", "content": "Greeting! I am an AI research assistant. How can I help you today?"},
        {"role": "user", "content": "Can you tell me about the creation of black holes?"}
    ],
    temperature=0,
)
```

LangChain supports these different types of messages, including a arbitrary role parameter (`ChatMessage`). Let's try: 

In [7]:
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='positive')

Let's try an example that involves a system instruction and a task provided by user.

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are positive, negative and neutral."),
    HumanMessage(content="Classify the following sentence: I am doing brilliant today!"),
]

chat.invoke(messages)

AIMessage(content='The sentence "I am doing brilliant today!" can be classified as positive sentiment.')

Now let's try another example that involves an exchange between a human and AI research assistant:

In [9]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat.invoke(messages)

AIMessage(content="Certainly! Black holes are fascinating astronomical objects that form from the remnants of massive stars. The creation of a black hole occurs through a process known as stellar collapse.\n\nWhen a massive star exhausts its nuclear fuel, it can no longer sustain the outward pressure generated by nuclear fusion. As a result, the star's core collapses under the force of gravity. This collapse is triggered by the imbalance between the inward gravitational force and the outward pressure.\n\nDuring the collapse, the star's core becomes incredibly dense, packing an enormous amount of mass into a tiny volume. This extreme density leads to the formation of a singularity, a point of infinite density at the center of the black hole.\n\nSurrounding the singularity is the event horizon, which is the boundary beyond which nothing, not even light, can escape the gravitational pull of the black hole. The event horizon is determined by the mass of the black hole, with larger black ho

There is even a feature to batch these requests and generate response (using `chat.response()`) like so:

In [10]:
batch_messages = [
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you tell me about the creation of black holes?")
    ],
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you explain the dark matter?")
    ]
]

chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text="Certainly! Black holes are fascinating astronomical objects that are formed through the gravitational collapse of massive stars. The creation of a black hole occurs when a star exhausts its nuclear fuel and can no longer sustain the outward pressure generated by nuclear fusion in its core.\n\nWhen a massive star reaches the end of its life, it undergoes a supernova explosion. During this explosive event, the outer layers of the star are ejected into space, while the core collapses inward due to its own gravity. If the core's mass is above a certain threshold, known as the Tolman-Oppenheimer-Volkoff (TOV) limit, the collapse continues until it forms a black hole.\n\nThe core collapse is an incredibly dense and energetic process. As the core collapses, its mass becomes concentrated into an extremely small volume, resulting in a gravitational field so strong that not even light can escape from it. This region of space is known as the event hori

If you look at the examples above it might be easier to just use a prompt template. LangChain also supports. Let's try that below:

In [11]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())

AIMessage(content='positive')

In [13]:
chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='neutral')